# Computer Assignment 1 (Search)

In [34]:
from timeit import default_timer as timer
from copy import deepcopy, copy
from typing import Callable, Union
from collections import deque
import heapq

In [35]:
INPUT_COUNT = 5
INPUT_PATH  = 'data/input%d.txt'
TIMER_TEST_COUNT = 3

In [36]:
class Graph:
    n: int
    m: int
    current: int
    cost: int
    parent: Union['Graph', None]
    adj: list[set[int]]
    difficulties: dict[int, int]
    requirements: dict[int, set[int]]
    remainingTime: int
    
    def __init__(self, n: int, m: int):
        self.n = n
        self.m = m
        self.current = 0
        self.cost = 0
        self.parent = None
        self.adj = [set() for _ in range(n)]
        self.difficulties = {}
        self.requirements = {}
        self.remainingTime = 0

    def __deepcopy__(self, memo: dict) -> 'Graph':
        g = Graph(self.n, self.m)
        g.current = self.current
        g.cost = self.cost
        g.parent = self.parent
        g.adj = copy(self.adj)
        g.difficulties = deepcopy(self.difficulties)
        g.requirements = deepcopy(self.requirements)
        g.remainingTime = self.remainingTime
        return g

    def __lt__(self, other: 'Graph') -> bool:
        return self.cost < other.cost

    def __key(self) -> tuple[int, dict[int, int], dict[int, set[int]]]:
        return (self.current, self.difficulties, self.requirements)

    def __str__(self) -> str:
        return str(self.__key())

    def __hash__(self) -> int:
        return hash(self.__str__())

Each state is defined as a graph which contains a list of remaining devotees, a list of remaining recipes, sayid's current location, the status of passage of difficult peaks, and the remaining time to get out of the current location (if it is a difficult peak). Also, the following values are stored in each state:

- `cost`: the cost of the path from the initial state to the current state
- `parent`: the parent state of the current state
- `edges`: the list of edges in the map

We consider to states as equal if the following values are equal in both states:

- sayid's current location
- remaining devotees and remaining recipes for each devotee
- the status of passage of difficult peaks

Goal state is defined as a state in which there exists no remaining devotees and no remaining recipes.  

The actions can be one of the following:

- decrease the remaining time of the current location by 1 if it is a difficult peak
- move to an adjacent location and memorize any recipe that is found there and satisfy any devotee that is found there if possible  

Initial state is the input graph with all the values given in the input file.

In [37]:
def readInput(path: str) -> Graph:
    with open(path, 'r', encoding='utf-8') as f:
        n, m = map(int, f.readline().split())
        graph = Graph(n, m)
        for _ in range(m):
            u, v = map(int, f.readline().split())
            graph.adj[u - 1].add(v - 1)
            graph.adj[v - 1].add(u - 1)
        _ = int(f.readline())
        difficulties = map(int, f.readline().split())
        for d in difficulties:
            graph.difficulties[d - 1] = -1
        s = int(f.readline())
        for _ in range(s):
            inp = f.readline().split()
            p = int(inp[0]) - 1
            vertices = {int(x) - 1 for x in inp[2:]}
            graph.requirements[p] = vertices
        i = int(f.readline()) - 1
        graph.current = i
        if i in graph.difficulties:
            graph.difficulties[i] += 1
        for u, vertices in graph.requirements.items():
            if i in vertices:
                vertices.remove(i)
        if i in graph.requirements and len(graph.requirements[i]) == 0:
            del graph.requirements[i]
    return graph

In [38]:
inputs = [readInput(INPUT_PATH % i) for i in range(1, INPUT_COUNT + 1)]

Inputs are given in `data` folder and are in `input#.txt` format. They are as follows:

- `input1.txt`: the first test case in both simple and main tests
- `input2.txt`: the second test case in simple tests
- `input3.txt`: the third test case in simple tests
- `input4.txt`: the second test case in main tests
- `input5.txt`: the third test case in main tests

In [39]:
def searchTime(g: Graph, method: Callable[[Graph], tuple[list[int] | None, int | None, int]]) -> float:
    sum = 0
    for _ in range(TIMER_TEST_COUNT):
        gCopy = deepcopy(g)
        start = timer()
        method(gCopy)
        end = timer()
        sum += end - start
    return sum / TIMER_TEST_COUNT

In [40]:
def moveInGraph(g: Graph, dest: int) -> Graph | None:
    if dest not in g.adj[g.current]:
        return None
    newGraph = deepcopy(g)
    newGraph.current = dest
    newGraph.cost += 1
    newGraph.parent = g
    if dest in newGraph.difficulties:
        newGraph.difficulties[dest] += 1
        newGraph.remainingTime = newGraph.difficulties[dest]
    for _, vertices in newGraph.requirements.items():
        if dest in vertices:
            vertices.remove(dest)
    if dest in newGraph.requirements and len(newGraph.requirements[dest]) == 0:
        newGraph.requirements.pop(dest)
    return newGraph


def isGoal(g: Graph) -> bool:
    return len(g.requirements) == 0


def getPath(g: Graph | None) -> list[int]:
    path = []
    while g is not None:
        path.append(g.current + 1)
        g = g.parent
    path.reverse()
    return path

In [41]:
def bfs(g: Graph) -> tuple[list[int] | None, int | None, int]:
    if isGoal(g):
        return getPath(g), g.cost,1
    discovered = set()
    queue = deque([g])
    discovered.add(str(g))
    while queue:
        g = queue.popleft()
        if g.remainingTime > 0:
            g.remainingTime -= 1
            g.cost += 1
            queue.append(g)
            continue
        for dest in g.adj[g.current]:
            newState = moveInGraph(g, dest)
            if newState is None:
                continue
            if isGoal(newState):
                return getPath(newState), newState.cost, len(discovered)
            if str(newState) not in discovered:
                queue.append(newState)
                discovered.add(str(newState))
    return None, None, len(discovered)

We know that passing an edge will take 1 second. If we are stuck in a difficult peak and `n` seconds are remaining to get out of it, we will expand the current state `n` times. For example, if we are stuck in a difficult peak and 5 seconds are left to get out of it, the current state will be repeated for 5 depths in the search tree, and in the $6^{th}$ depth, we can move to an adjacent location. This method results in not choosing a wrong path through a difficult peak. As a result, the `BFS` algorithm will be able to find the optimal solution.

In [57]:
for i, g in enumerate(inputs):
    gCopy = deepcopy(g)
    print(f'BFS: Input {i + 1}')
    path, cost, stateCount = bfs(gCopy)
    if path is not None:
        print('Path:', ' -> '.join(map(str, path)))
        print('Cost:', cost)
        print('Total Visited States:', stateCount)
        print(f'Time: {searchTime(g, bfs):.4f} seconds')
    else:
        print('No solution')
    print()

BFS: Input 1
Path: 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Cost: 8
Total Visited States: 78
Time: 0.0069 seconds

BFS: Input 2
Path: 9 -> 10 -> 9 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Cost: 8
Total Visited States: 115
Time: 0.0098 seconds

BFS: Input 3
Path: 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
Cost: 13
Total Visited States: 1769
Time: 0.1697 seconds

BFS: Input 4
Path: 28 -> 19 -> 13 -> 3 -> 11 -> 24 -> 9 -> 2 -> 5 -> 7 -> 29 -> 22 -> 28
Cost: 12
Total Visited States: 7536
Time: 1.5185 seconds

BFS: Input 5
Path: 40 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 41 -> 18 -> 1 -> 19 -> 43 -> 49 -> 47 -> 49 -> 9 -> 34 -> 25 -> 50 -> 12 -> 16
Cost: 21
Total Visited States: 10630
Time: 1.2077 seconds



The run time of the algorithm is much less than the time limit.

In [43]:
def dfs(g: Graph, depth: int, discovered: set, visited: set) -> Graph | None:
    if isGoal(g):
        return g
    if depth <= 0:
        return None
    remainingTime = g.remainingTime
    if remainingTime > 0:
        g.remainingTime = 0
        g.cost += remainingTime
        if depth <= remainingTime + 1:
            return None
    children = set()
    for dest in g.adj[g.current]:
        newState = moveInGraph(g, dest)
        if newState is None:
            continue
        if str(newState) in visited:
            continue
        children.add(((newState, str(newState))))
        discovered.add(str(newState))
        visited.add(str(newState))
    for child, _ in children:
        ans = dfs(child, depth - 1 - remainingTime, discovered, visited)
        if ans is not None:
            return ans
    
    for _, key in children:
        visited.remove(str(key))
    return None


def ids(g: Graph) -> tuple[list[int] | None, int | None, int]:
    depth = 0
    while True:
        discovered = set()
        discovered.add(str(g))
        visited = set()
        visited.add(str(g))
        ans = dfs(g, depth, discovered, visited)
        if ans is not None:
            return getPath(ans), ans.cost, len(discovered)
        # print(f'IDS: Depth {depth} - Visited States: {len(discovered)}')
        depth += 1

The `DFS` algorithm will not give us the optimal solution because a better path may be found later in the search tree. However, when we use the `IDS` algorithm, we will be able to find the optimal solution. his is because the `IDS` algorithm will search the tree in depth-first manner, but it will limit the depth of the search tree. As a result, if a better path has existed in the tree, it would have been found in the previous depths  

This algorithm will visit many states (increasing exponentially with the depth of the search tree) and only a small percentage of them are not repeated.  

If we uncomment the `print` statements in the code, the number of non-repeated states will be shown. Also, if we change the type of `discovered` from `set` to `list`, the total number of states will be shown. These numbers are presented below for `input4.txt`:

- Depth = 6 : 90k total states, 1k non-repeated states
- Depth = 7 : 660k total states, 2k non-repeated states
- Depth = 8 : 4.8M total states, 3k non-repeated states

If we continue to increase the depth, the number of total states will be more than 5 billion in the $12^{th}$ depth. As a result, the algorithm will take a long time to find the optimal solution for `input4.txt` and `input5.txt`.

In [58]:
for i, g in enumerate(inputs[:3]): # remove [:3] to run all inputs (takes a long time =] )
    gCopy = deepcopy(g)
    print(f'IDS: Input {i + 1}')
    path, cost, stateCount = ids(gCopy)
    if path is not None:
        print('Path:', ' -> '.join(map(str, path)))
        print('Cost:', cost)
        print('Total Visited States:', stateCount)
        print(f'Time: {searchTime(g, ids):.4f} seconds')
    else:
        print('No solution')
    print()

IDS: Input 1
Path: 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Cost: 8
Total Visited States: 76
Time: 0.0135 seconds

IDS: Input 2
Path: 9 -> 10 -> 2 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Cost: 8
Total Visited States: 88
Time: 0.0541 seconds

IDS: Input 3
Path: 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
Cost: 13
Total Visited States: 1521
Time: 1.5436 seconds



Despite the fact that no limit is mentioned in the problem for `IDS` algorithm, the algorithm takes a reasonable amount of time to find the optimal solution for simple tests.  

I decided to run the algorithm for `input4.txt` and `input5.txt` in a private server with 1 core and 2 GB RAM. The results are presented in the following image:

![IDS](./assets/IDS%20Results.png)

In [50]:
def heuristic(g: Graph) -> int:
    remainingCount = set()
    for r, vertices in g.requirements.items():
        remainingCount.add(r)
        remainingCount.update(vertices)
    return len(remainingCount)

def aStar(g: Graph, alpha: float = 1) -> tuple[list[int] | None, int | None, int]:
    q = []
    heapq.heappush(q, (0, g))
    discovered = set()
    discovered.add(str(g))
    while q:
        _, g = heapq.heappop(q)
        if isGoal(g):
            return getPath(g), g.cost, len(discovered)
        if g.remainingTime > 0:
            g.remainingTime -= 1
            g.cost += 1
            heapq.heappush(q, (g.cost + alpha * heuristic(g), g))
            continue
        for dest in g.adj[g.current]:
            newState = moveInGraph(g, dest)
            if newState is None:
                continue
            if str(newState) not in discovered:
                heapq.heappush(q, (newState.cost + alpha * heuristic(newState), newState))
                discovered.add(str(newState))
    return None, None, len(discovered)

For heuristic, I consider the number of remaining devotees and the number of remaining recipes. However, it is important to note that these values will be considered for unique vertices. For example, if there are 2 devotees in a vertex, and there are no recipes left, the heuristic value will be 1.  

Proving the consistency of the heuristic:  

Consider two states `A` and `C` in which `C` is one of the grandchildren of `A`. Also, consider the number of remaining vertices (devotees and recipes unique vertices) in `A` and `C` to be `n` and `m` respectively. In this case, we should pass at least `n-m` edges to reach `C` from `A`. As a result, We know that passing each edge will take 1 second. Therefore, we can say: $$cost(A\ to\ C) \geq n-m = h(A) - h(C)$$

As a result, the heuristic is consistent.  

Also, we could use `MST` algorithm as a heuristic. In this case, for each state, we will find the minimum spanning tree of the remaining vertices. Although this is a better heuristic, it will take a long time.  

`A*` algorithm will be able to find the optimal solution if the following conditions are met:

- The heuristic is consistent
- Goal test should be done when popping a state from the frontier

As both conditions are met, `A*` algorithm will be able to find the optimal solution.

In [49]:
for i, g in enumerate(inputs):
    gCopy = deepcopy(g)
    print(f'A*: Input {i + 1}')
    path, cost, stateCount = aStar(gCopy)
    if path is not None:
        print('Path:', ' -> '.join(map(str, path)))
        print('Cost:', cost)
        print('Total Visited States:', stateCount)
        print(f'Time: {searchTime(g, lambda g: aStar(g, 1)):.4f} seconds')
    else:
        print('No solution')
    print()

A*: Input 1
Path: 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Cost: 8
Total Visited States: 78
Time: 0.0028 seconds

A*: Input 2
Path: 9 -> 10 -> 2 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Cost: 8
Total Visited States: 88
Time: 0.0089 seconds

A*: Input 3
Path: 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
Cost: 13
Total Visited States: 711
Time: 0.0576 seconds

A*: Input 4
Path: 28 -> 19 -> 13 -> 3 -> 11 -> 24 -> 9 -> 27 -> 5 -> 7 -> 29 -> 22 -> 28
Cost: 12
Total Visited States: 2670
Time: 0.4009 seconds

A*: Input 5
Path: 40 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 41 -> 18 -> 1 -> 19 -> 43 -> 49 -> 47 -> 49 -> 9 -> 34 -> 25 -> 50 -> 12 -> 16
Cost: 21
Total Visited States: 7894
Time: 1.1771 seconds



As it can be seen above, the `A*` algorithm has a great performance. It is able to find the optimal solution for all tests in a reasonable amount of time (less than the time limit).  

All of the algorithms that are used until now, will be able to find the optimal solution. However, the `A*` algorithm has the best performance.  

The next difference is the memory usage. The `BFS` and `A*` algorithms use exponential memory. On the other hand, the `IDS` algorithm uses linear memory. In this case, we will have no problem with the memory usage.

In [52]:
for i, g in enumerate(inputs):
    gCopy = deepcopy(g)
    print(f'A* (alpha = 1.2): Input {i + 1}')
    path, cost, stateCount = aStar(gCopy, 1.2)
    if path is not None:
        print('Path:', ' -> '.join(map(str, path)))
        print('Cost:', cost)
        print('Total Visited States:', stateCount)
        print(f'Time: {searchTime(g, lambda g: aStar(g, 2)):.4f} seconds')
    else:
        print('No solution')
    print()

A* (alpha = 1.2): Input 1
Path: 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Cost: 8
Total Visited States: 52
Time: 0.0012 seconds

A* (alpha = 1.2): Input 2
Path: 9 -> 10 -> 2 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Cost: 8
Total Visited States: 55
Time: 0.0037 seconds

A* (alpha = 1.2): Input 3
Path: 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
Cost: 13
Total Visited States: 425
Time: 0.0089 seconds

A* (alpha = 1.2): Input 4
Path: 28 -> 23 -> 9 -> 24 -> 11 -> 3 -> 13 -> 23 -> 5 -> 7 -> 29 -> 22 -> 28
Cost: 12
Total Visited States: 436
Time: 0.0055 seconds

A* (alpha = 1.2): Input 5
Path: 40 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 41 -> 18 -> 1 -> 19 -> 43 -> 49 -> 47 -> 49 -> 9 -> 34 -> 25 -> 50 -> 12 -> 16
Cost: 21
Total Visited States: 6565
Time: 0.3648 seconds



The number of visited states and the run time of the `Weighted A*` algorithm is much less than the `A*` algorithm. This is because the heuristic will be multiplied by a constant value in order to make it more realistic.  

In this test, the `Weighted A*` algorithm has given us the optimal solution. However, it is not guaranteed that it will give us the optimal solution in all cases.

In [51]:
for i, g in enumerate(inputs):
    gCopy = deepcopy(g)
    print(f'A* (alpha = 1.4): Input {i + 1}')
    path, cost, stateCount = aStar(gCopy, 1.4)
    if path is not None:
        print('Path:', ' -> '.join(map(str, path)))
        print('Cost:', cost)
        print('Total Visited States:', stateCount)
        print(f'Time: {searchTime(g, lambda g: aStar(g, 2)):.4f} seconds')
    else:
        print('No solution')
    print()

A* (alpha = 1.4): Input 1
Path: 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Cost: 8
Total Visited States: 52
Time: 0.0021 seconds

A* (alpha = 1.4): Input 2
Path: 9 -> 10 -> 2 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Cost: 8
Total Visited States: 46
Time: 0.0015 seconds

A* (alpha = 1.4): Input 3
Path: 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
Cost: 13
Total Visited States: 254
Time: 0.0084 seconds

A* (alpha = 1.4): Input 4
Path: 28 -> 19 -> 3 -> 11 -> 24 -> 9 -> 2 -> 5 -> 7 -> 29 -> 20 -> 13 -> 23 -> 28
Cost: 13
Total Visited States: 213
Time: 0.0052 seconds

A* (alpha = 1.4): Input 5
Path: 40 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 41 -> 18 -> 1 -> 19 -> 43 -> 49 -> 47 -> 49 -> 9 -> 34 -> 25 -> 50 -> 12 -> 16
Cost: 21
Total Visited States: 5752
Time: 0.4214 seconds



As it is shown above, the `Weighted A*` algorithm has not given us the optimal solution for $\alpha = 1.4$. In `input4.txt`, the algorithm has found a solution with a cost of 13, while the optimal solution has a cost of 12.  

In conclusion, the `Weighted A*` can have a great performance if the value of $\alpha$ is chosen properly. However, it is not guaranteed that it will give us the optimal solution in all cases.  

The results are shown in the following tables:

## input1.txt

|                            | Minimum Cost | Number of Visited States | Execution Time |
|         :----:             |    :----:    |          :----:          |     :----:     |
|          BFS               |      8       |            78            |     0.0069s    |
|          IDS               |      8       |            76            |     0.0135s    |
|           A*               |      8       |            78            |     0.0028s    |
| Weighted A* $(\alpha=1.2)$ |      8       |            52            |     0.0012s    |
| Weighted A* $(\alpha=1.4)$ |      8       |            52            |     0.0021s    |

## input2.txt

|                            | Minimum Cost | Number of Visited States | Execution Time |
|         :----:             |    :----:    |          :----:          |     :----:     |
|          BFS               |      8       |           115            |     0.0098s    |
|          IDS               |      8       |            88            |     0.0541s    |
|           A*               |      8       |            88            |     0.0089s    |
| Weighted A* $(\alpha=1.2)$ |      8       |            55            |     0.0037s    |
| Weighted A* $(\alpha=1.4)$ |      8       |            46            |     0.0015s    |

## input3.txt

|                            | Minimum Cost | Number of Visited States | Execution Time |
|         :----:             |    :----:    |          :----:          |     :----:     |
|          BFS               |      13      |           1769           |     0.1697s    |
|          IDS               |      13      |           1521           |     1.5436s    |
|           A*               |      13      |           711            |     0.0576s    |
| Weighted A* $(\alpha=1.2)$ |      13      |           425            |     0.0089s    |
| Weighted A* $(\alpha=1.4)$ |      13      |           254            |     0.0084s    |

## input4.txt

|                            | Minimum Cost | Number of Visited States | Execution Time |
|         :----:             |    :----:    |          :----:          |     :----:     |
|          BFS               |      12      |           7536           |     1.5185s    |
|          IDS               |      12      |           6266           |    1h42m22s    |
|           A*               |      12      |           2670           |     0.4009s    |
| Weighted A* $(\alpha=1.2)$ |      12      |           436            |     0.0055s    |
| Weighted A* $(\alpha=1.4)$ |      13      |           213            |     0.0052s    |

## input5.txt

|                            | Minimum Cost | Number of Visited States | Execution Time |
|         :----:             |    :----:    |          :----:          |     :----:     |
|          BFS               |      21      |           10630          |     1.2077s    |
|          IDS               |      21      |           10465          |    14h24m39s   |
|           A*               |      21      |           7894           |     1.1771s    |
| Weighted A* $(\alpha=1.2)$ |      21      |           6565           |     0.3648s    |
| Weighted A* $(\alpha=1.4)$ |      21      |           5752           |     0.4214s    |